# DAT 19: Homework 2 Assignment

## Instructions

For Homework 2, we will build on the work we did with the Titanic dataset in Homework 1. In this assignment, we will build a logistic regression model to predict passenger survival.

Please do all your analysis to answer the questions below in this Jupyter notebook. Show your work.

**Please submit your completed notebook by 6:00PM on Monday, January 11.**

## About the Data

```
VARIABLE DESCRIPTIONS:
survival        Survival
                (0 = No; 1 = Yes)
pclass          Passenger Class
                (1 = 1st; 2 = 2nd; 3 = 3rd)
name            Name
sex             Sex
age             Age
sibsp           Number of Siblings/Spouses Aboard
parch           Number of Parents/Children Aboard
ticket          Ticket Number
fare            Passenger Fare
cabin           Cabin
embarked        Port of Embarkation
                (C = Cherbourg; Q = Queenstown; S = Southampton)

SPECIAL NOTES:
Pclass is a proxy for socio-economic status (SES)
 1st ~ Upper; 2nd ~ Middle; 3rd ~ Lower

Age is in Years; Fractional if Age less than One (1)
 If the Age is Estimated, it is in the form xx.5

With respect to the family relation variables (i.e. sibsp and parch)
some relations were ignored.  The following are the definitions used
for sibsp and parch.

Sibling:  Brother, Sister, Stepbrother, or Stepsister of Passenger Aboard Titanic
Spouse:   Husband or Wife of Passenger Aboard Titanic (Mistresses and Fiances Ignored)
Parent:   Mother or Father of Passenger Aboard Titanic
Child:    Son, Daughter, Stepson, or Stepdaughter of Passenger Aboard Titanic

Other family relatives excluded from this study include cousins,
nephews/nieces, aunts/uncles, and in-laws.  Some children travelled
only with a nanny, therefore parch=0 for them.  As well, some
travelled with very close friends or neighbors in a village, however,
the definitions do not support such relations.
```

## Homework Assignment

**1) Create a logistic regression model on the Titanic dataset to predict the survival of passengers. Show your model output. Include coefficient values.**

In [96]:
import csv as csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



data_raw = pd.read_csv('titanic.csv')
data_raw.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,NaN,S


In [97]:
data_raw.columns

Index([u'PassengerId', u'Survived', u'Pclass', u'Name', u'Sex', u'Age',
       u'SibSp', u'Parch', u'Ticket', u'Fare', u'Cabin', u'Embarked'],
      dtype='object')

In [98]:
from sklearn.preprocessing import LabelEncoder

enc_lab={}
# adjust the cabin to the first letter in the string
data_raw['Cabin_adj']=data_raw['Cabin'].str[0]
col_adj=['Sex', 'Embarked', 'Cabin_adj']

data=data_raw.copy(deep=True)

# apply label encoder
for i in data_raw[col_adj]:
    enc_lab[i]=LabelEncoder()
    data[i]=enc_lab[i].fit_transform(data_raw[i])

    
# remove all nan's
data=data.fillna(0)
y_train=data.values[:,1]
y_train=map(int, np.array(y_train).tolist())
X_train=data.drop(['Name', 'Cabin', 'Ticket'], axis=1).values[:, 2:]

data.drop(['Name', 'Cabin', 'Ticket'], axis=1).columns

Index([u'PassengerId', u'Survived', u'Pclass', u'Sex', u'Age', u'SibSp',
       u'Parch', u'Fare', u'Embarked', u'Cabin_adj'],
      dtype='object')

In [99]:
data_raw.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Cabin_adj
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,NaN,S,NaN
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,NaN,S,NaN
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S,C
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,NaN,S,NaN


In [126]:
%matplotlib inline
# data.hist(figsize=(8,8))
from sklearn.linear_model import LogisticRegression
clf=LogisticRegression(C=1)
clf.fit(X_train, y_train)
print 'Logit Coefficient values ', zip(data.drop(['Name', 'Cabin', 'Ticket'], axis=1).columns[2:], clf.coef_.ravel())


Logit Coefficient values  [('Pclass', -0.60662167341299145), ('Sex', -2.5312512776240812), ('Age', -0.013357886313315319), ('SibSp', -0.26340813497599941), ('Parch', -0.056808021925159199), ('Fare', 0.0037424819909905923), ('Embarked', -0.12076923053899824), ('Cabin_adj', 0.20129772879608387)]


**2) Which features are predictive for this logistic regression? Explain your thinking. Do not simply cite model statistics.**

Sex and passenger class seem like the most predictive based on the coefficient values. This makes sense because as the idea of saving females first was a very normal idea back then. Passenger class also seems very likely as a predictor as the class would indicate their location on the boat as well as access to exits in order to reach the lifeboats.

**3) Implement cross-validation for your logistic regression model. Select the number of folds. Explain your choice.**

In [124]:
from sklearn.cross_validation import cross_val_score
# len(y_train)
cv_sizes=[5, 10, 25, 50]
for i in cv_sizes:
    print 'CV size', i, 'mean score', cross_val_score(clf, X_train, y_train, cv=i).mean()





CV size 5 mean score 0.794630977446
CV size 10 mean score 0.797972420838
CV size 25 mean score 0.801449112979
CV size 50 mean score 0.798071895425


In [ ]:
I choose 25 folds since it has the highest mean cross val score

**4) In the hw-assignments director on the class github repo, there is a file called titanic-test.csv. What does your logistic regression model predict for these previously unseen (i.e. out of sample) passengers?**

In [133]:
test_raw=pd.read_csv('titanic-test.csv')
# adjust the cabin to the first letter in the string
test_raw['Cabin_adj']=test_raw.Cabin.str[0]

test=test_raw.copy(deep=True)
# apply label encoder
for i in test_raw[col_adj]:
    test[i]=enc_lab[i].transform(test_raw[i])

# remove all nan's
test=test.fillna(0)
# print test.head()
X_test=test.drop(['Cabin', 'Name', 'Ticket'], axis=1).values[:,1:]

pred=clf.predict(X_test)

print (map(int, np.array(pred).tolist()))



[0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 

418